In [1]:
!pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached regex-2024.11.6-cp39-cp39-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp39-cp39-win_amd64.whl (274 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [214]:
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,f1_score
import nltk
import string

In [215]:
dataset=pd.read_csv("C:\Projects\Research_paper_classifier\Research-Paper-Evaluater\IIT KHARAGPUR DATASET.csv")
dataset.head(10)

,Paper name,Content,label
0,P001.pdf,Transdimensional Properties of Graphite in Rel...,0
1,P002.pdf,Synergistic Convergence of Photosynthetic Path...,0
2,P003.pdf,Deciphering the Enigmatic Properties of Metals...,0
3,P004.pdf,AI-Driven Personalization in Online Education ...,0
4,P005.pdf,Analyzing Real-Time Group Coordination in Augm...,0
5,P006.pdf,Detailed Action Identification in Baseball Gam...,1
6,P007.pdf,Advancements in 3D Food Modeling: A Review of ...,1
7,P008.pdf,Advanced techniques for through and contextual...,1
8,P009.pdf,The Importance of Written Explanations in Aggr...,1
9,P010.pdf,Detecting Medication Usage in Parkinson’s Dise...,1


In [216]:
dataset.isnull().sum()

Paper name    0
Content       0
label         0
dtype: int64

In [217]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("punkt_tab")
stop_words=set(stopwords.words('english'))
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91630\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\91630\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [218]:
def rmv_stopwords(text):
    if isinstance(text,str):

     tokens=word_tokenize(text)
     filtered_tokens=set(token for token in tokens if token.lower() not in stop_words)
     return ' '.join(filtered_tokens)
    return text

In [219]:
def rmv_punctuation(text):
    if isinstance(text,str):
        filtered_text=[word for word in text if word not in string.punctuation]
        return ''.join(filtered_text)
    return text

In [220]:
dataset["Content"]=dataset["Content"].apply(lambda x:rmv_punctuation(x))

In [221]:
dataset["Content"]=dataset["Content"].apply(rmv_stopwords)

In [222]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
x=tfidf.fit_transform(dataset["Content"]).toarray()
y=dataset['label']

In [223]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)


In [224]:
param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
param_grid_lr= {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 500]
}
param_grid_dt={
    'criterion' : ["gini", "entropy", "log_loss"],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': [None, 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20, 50],
    'ccp_alpha': [0.0, 0.01, 0.1]
}
param_grid_svc = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto'],
}
param_grid_naive={
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}


In [225]:
gb=GradientBoostingClassifier(random_state=42)
rf=RandomForestClassifier(random_state=42)
lr=LogisticRegression(random_state=42)
dt=DecisionTreeClassifier(random_state=42,)
svc=SVC(random_state=42)
gnb=GaussianNB()

In [226]:
grid_gb=GridSearchCV(gb,param_grid=param_grid_gb,cv=3,scoring='accuracy',verbose=1,n_jobs=-1)

In [227]:
grid_rf=GridSearchCV(rf,param_grid_rf,cv=3,scoring='accuracy',verbose=1,n_jobs=-1)

In [228]:
grid_lr=GridSearchCV(lr,param_grid_lr,cv=3,scoring='accuracy',verbose=1,n_jobs=-1)

In [229]:
grid_dt=GridSearchCV(dt,param_grid_dt,cv=3,scoring='accuracy',verbose=1,n_jobs=-1)

In [230]:
grid_svc=GridSearchCV(svc,param_grid_svc,cv=3,scoring='accuracy',verbose=1,n_jobs=-1)

In [231]:
grid_gnb=GridSearchCV(gnb,param_grid_naive,cv=3,scoring='accuracy',verbose=1,n_jobs=-1)

In [232]:
grid_gb.fit(x_train,y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=1)

In [233]:
grid_rf.fit(x_train,y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=1)

In [234]:
grid_lr.fit(x_train,y_train)

Fitting 3 folds for each of 120 candidates, totalling 360 fits


c:\Users\91630\anaconda3\envs\Researchpaper\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\91630\anaconda3\envs\Researchpaper\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\91630\anaconda3\envs\Researchpaper\lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\91630\anaconda3\envs\Researchpaper\lib\site-packages\sklearn\linear_model\_logistic.py", line

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'max_iter': [100, 200, 500],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['liblinear', 'saga']},
             scoring='accuracy', verbose=1)

In [235]:
grid_dt.fit(x_train,y_train)

Fitting 3 folds for each of 10368 candidates, totalling 31104 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'ccp_alpha': [0.0, 0.01, 0.1],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 5, 10, 20],
                         'max_features': [None, 'sqrt', 'log2'],
                         'max_leaf_nodes': [None, 10, 20, 50],
                         'min_samples_leaf': [1, 2, 5, 10],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']},
             scoring='accuracy', verbose=1)

In [236]:
grid_svc.fit(x_train,y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


GridSearchCV(cv=3, estimator=SVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'degree': [2, 3, 4],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='accuracy', verbose=1)

In [237]:
grid_gnb.fit(x_train,y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


GridSearchCV(cv=3, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]},
             scoring='accuracy', verbose=1)

In [238]:
best_param_gb=grid_gb.best_estimator_
best_param_rf=grid_rf.best_estimator_
best_param_lr=grid_lr.best_estimator_
best_param_dt=grid_dt.best_estimator_
best_param_svc=grid_svc.best_estimator_
best_param_gnb=grid_gnb.best_estimator_

In [239]:
y_pred_gb=best_param_gb.predict(x_test)

In [240]:
y_pred_rf=best_param_rf.predict(x_test)

In [241]:
y_pred_lr=best_param_lr.predict(x_test)

In [242]:
y_pred_dt=best_param_dt.predict(x_test)

In [243]:
y_pred_svc=best_param_svc.predict(x_test)

In [244]:
y_pred_gnb=best_param_gnb.predict(x_test)

In [245]:
print(f"accuracy GB:{accuracy_score(y_test,y_pred_gb)}")
print(f"accuracy RF:{accuracy_score(y_test,y_pred_rf)}")
print(f"accuracy LR:{accuracy_score(y_test,y_pred_lr)}")
print(f"accuracy DT:{accuracy_score(y_test,y_pred_dt)}")
print(f"accuracy SVC:{accuracy_score(y_test,y_pred_svc)}")
print(f"accuracy GNB:{accuracy_score(y_test,y_pred_gnb)}")



accuracy GB:0.37037037037037035
accuracy RF:0.4444444444444444
accuracy LR:0.4444444444444444
accuracy DT:0.37037037037037035
accuracy SVC:0.4444444444444444
accuracy GNB:0.4444444444444444


In [246]:

print(f"f1_score GB:{f1_score(y_true=y_test,y_pred=y_pred_gb)}")
print(f"f1_score RF:{f1_score(y_true=y_test,y_pred=y_pred_rf)}")
print(f"accuracy LR:{f1_score(y_true=y_test,y_pred=y_pred_lr)}")
print(f"accuracy DT:{f1_score(y_true=y_test,y_pred=y_pred_dt)}")
print(f"accuracy SVC:{f1_score(y_true=y_test,y_pred=y_pred_svc)}")
print(f"accuracy GNB:{f1_score(y_true=y_test,y_pred=y_pred_gnb)}")


f1_score GB:0.48484848484848486
f1_score RF:0.34782608695652173
accuracy LR:0.0
accuracy DT:0.45161290322580644
accuracy SVC:0.0
accuracy GNB:0.21052631578947367
